# NCSU GIS 582: Geospatial Modeling and Analysis

## 3A - Geospatial Analysis I: global, zonal and focal operations, map algebra

### Resources

- [GRASS Manual](https://grass.osgeo.org/grass-devel/manuals/)
- [GRASS Jupyter Introduction](https://grass.osgeo.org/grass-devel/manuals/jupyter_intro.html)
- [GRASS Python API Introduction](https://grass.osgeo.org/grass-devel/manuals/python_intro.html)

## Google Colab Setup

Let’s first print system description to know where are we.

In [ ]:
!lsb_release -a

At the time of writing this tutorial, Colab has Linux Ubuntu 22.04.4 LTS. So we add the ppa:ubuntugis repository, update and install GRASS. It might take a couple of minutes according to the resources available.

In [ ]:
!add-apt-repository -y ppa:ubuntugis/ubuntugis-unstable
!apt update
!apt-get install -y grass-core grass-dev

Check that GRASS is installed by asking which version is there.

In [ ]:
!grass --version

Check which Python version is running.

In [ ]:
import sys

v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

## 1. Import Python Packages

Import the Python standard libraries we need.

In [ ]:
import subprocess
import os
from pathlib import Path

We are going to import the GRASS Python API (`grass.script`) and the GRASS Jupyter package (`grass.jupyter`), but first, we need to find the path to those packages using the `--config python_path` command. This command is slightly different for each operating system.

We use `subprocess.check_output` to find the path and `sys.path.append` to add it to the path.

In [ ]:
# Ask GRASS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

Import the GRASS packages we need.

In [ ]:
import grass.script as gs
import grass.jupyter as gj

Download and unzip the North Carolina basic sample dataset:

> A complete list of available datasets can be found at: https://grass.osgeo.org/download/data/

In [ ]:
!grass --tmp-project XY --exec g.download.project url=https://grass.osgeo.org/sampledata/north_carolina/nc_basic_spm_grass7.tar.gz path=$HOME

Start GRASS Session

In [ ]:
# Start GRASS Session
session = gj.init("~/grassdata", "nc_spm_08_grass7", "user1")

## Tutorial Tasks

### 1. Compute and map summarry statistics

Compute zonal statistics maps representing average slope for each basin, often needed for hydrologic modeling.

In [ ]:
!g.region raster=slope -p
!r.stats.zonal base=basin_50K cover=slope method=average output=slope_avgbasin
!r.colors slope_avgbasin color=gyr

Now display the results.

In [ ]:
slope_avgbasin_display = gj.Map(filename="slope_avgbasin.png")
slope_avgbasin_display.d_rast(map="slope_avgbasin")
slope_avgbasin_display.d_legend(raster="slope_avgbasin", at=[90,50,5,8])
slope_avgbasin_display.d_vect(map="streams", color="#0f196e")
slope_avgbasin_display.d_vect(map="streets_wake", color="grey")
slope_avgbasin_display.show()

Compute zonal statistics maps representing most common land use for each basin.
To get the best result, make sure you use all the flags from the example below.

In [ ]:
!g.region raster=landuse96_28m -p
!r.mode base=basin_50K cover=landuse96_28m output=landuse96_modebasin

In [ ]:
# Create Map instance
landuse96_modebasin_map = gj.Map(filename="landuse96_modebasin.png")
landuse96_modebasin_map.d_rast(map="landuse96_modebasin")
landuse96_modebasin_map.d_legend(
    raster="landuse96_modebasin",
    at=[40, 20, 2, 5],
    flags="nfc"
)
landuse96_modebasin_map.d_vect(map="streams", color="#0f196e")
landuse96_modebasin_map.show()

#### Question 1

**What is the most common land use in the two basins with the steepest average slope? You can use map query or map algebra to find out the naswer.**

> To query the map you cna use the `gj.InteractiveMap()` class to create an interactive map where you can click on features to get their values.

```python
m = gj.InteractiveMap()
m.add_raster("slope_avgbasin")
m.add_raster("landuse96_modebasin")
m.show()
```

### 2. Apply neighborhood operators and map algebra

#### Map land use diversity

1. Use neighborhood operator to compute land use diversity map and create a map of locations with the most homogeneous (single category) landuse where a land use diversification could be beneficial (e.g. by planting trees). 
2. Use 7x7 cells window to find areas which have single landuse within 200x200m.
3. Compute and display the land use diversity map. Adjust the legend size and placement of the scalebar and legend as needed.

In [ ]:
!g.region raster=landuse96_28m -p
!r.neighbors landuse96_28m output=lu_divers method=diversity size=7

Display the land use diversity map.

In [ ]:
# Create Map instance
lu_diversity_map_map = gj.Map(filename="lu_diversity_map.png")
lu_diversity_map_map.d_rast(map="lu_divers")
lu_diversity_map_map.d_legend(raster="lu_divers", at=[90,70,5,8], flags="v")
lu_diversity_map_map.d_vect(map="streets_wake", co="white")

# Display map
lu_diversity_map_map.show()

Report the results.

> Note: In [GRASS 8.5](https://grass.osgeo.org/grass85/manuals/r.report.html) support to output the report as [JSON](https://www.json.org/json-en.html) for easy parsing and ingesting into other analysis tools such as [Pandas DataFrame](https://pandas.pydata.org/).

In [ ]:
!r.report lu_divers unit=p

Use map algebra to extract the single category areas and find out the area totals for each category using the report tool.

In [ ]:
!r.mapcalc "landuse_1cat = if(lu_divers == 1, landuse96_28m, null())"
!r.colors landuse_1cat raster=landuse96_28m
!r.category landuse_1cat raster=landuse96_28m
!r.report landuse_1cat unit=h,p -n

Now display the results.

In [ ]:
# Create Map instance
lu_signle_map = gj.Map(filename="lu_signle_map.png")
lu_signle_map.d_rast(map="landuse_1cat")
lu_signle_map.d_legend(raster="landuse_1cat", flags="c")

# Display map
lu_signle_map.show()

#### Question 2

**Which land use covers the largest area with a single landuse category within 7x7 moving windows (most homogeneous areas)?**

**Use map algebra to extract a map layer that represents homogeneous "Mixed hardwoods / Conifers" areas (category 18) from the "landuse_1cat" layer that we may target for protection. Include the command and a resulting map with extracted map displayed together with "streets_wake" in your report**

### 3. Smoothing elevation raster and analyzing differences between DEMs

Use neighborhood operator to smooth the SRTM elevation map "elev_srtm_30m" and compare the summary statistics for the original and smoothed SRTM DEM. First, display the lidar-based "elev_ned_30m" map, assign the same colors to the "elev_srtm_30m" and compare their values using legends. Then use neighborhood operator applied to 5x5 window to smooth the "elev_srtm_30m" and compare the results using the r.univar tool. **How does the size of the neighborhood influence the resulting DEM?** (You can test different sizes yourself.)

In [ ]:
!g.region raster=elev_srtm_30m -p

In [ ]:
# Create Map instance
srtm_dem_original_map = gj.Map()
srtm_dem_original_map.d_rast(map="elev_srtm_30m")
srtm_dem_original_map.d_legend(raster="elev_srtm_30m")

# Display map
srtm_dem_original_map.show()

In [ ]:
!r.colors elev_srtm_30m raster=elevation

In [ ]:
# Create Map instance
srtm_dem_original_map = gj.Map(filename="srtm_dem_original.png")
srtm_dem_original_map.d_rast(map="elev_srtm_30m")
srtm_dem_original_map.d_legend(raster="elev_srtm_30m")

# Display map
srtm_dem_original_map.show()

Explore the univariate statistics of the original SRTM DEM.

In [ ]:
!r.univar elev_srtm_30m

Smooth the SRTM DEM using a 5x5 moving window.

In [ ]:
!r.neighbors elev_srtm_30m output=elev_srtm30m_sm5 method=average size=5

Display the smoothed SRTM DEM.

In [ ]:
# Create Map instance
srtm_dem_smoothed_map = gj.Map(filename="srtm_dem_smoothed.png")
srtm_dem_smoothed_map.d_rast(map="elev_srtm30m_sm5")
srtm_dem_smoothed_map.d_legend(raster="elev_srtm30m_sm5")
# Display map
srtm_dem_smoothed_map.show()

Compare univariate statistics of the smoothed DEM.

In [ ]:
!r.univar elev_srtm30m_sm5

Further explore the difference between the SRTM DEM "elev_srtm_30m" and lidar-based NED DEM "elev_ned_30m". First, compute the map of elevation differences using map algebra and find the range of differences:

In [ ]:
!r.mapcalc "srtm_ned30_dif = elev_srtm_30m - elev_ned_30m"
!r.info -r srtm_ned30_dif

Assign a divergent color table to distinguish the negative and positive values, [srtmneddiff_color.txt](https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/srtmneddiff_color.txt), and display the difference map with legend.

In [ ]:
# Create Map instance
srtm_dem_smoothed_map = gj.Map(filename="srtm_ned30_dif.png")
srtm_dem_smoothed_map.d_rast(map="srtm_ned30_dif")
srtm_dem_smoothed_map.d_legend(
    raster="srtm_ned30_dif",
    at=[90, 40, 5, 7]
)
# Display map
srtm_dem_smoothed_map.show()

#### Question 3

**Are the elevations in "elev_srtm_30m" higher in most areas or lower than in "elev_ned_30m"? Which result will you use to answer the above question - the "srtm_ned30_dif" map or the numbers provided by [r.univar](https://grass.osgeo.org/grass-stable/manuals/r.univar.html)? Are there any values in the elev_srtm_30m map that are not realistic? Where are they located?**

### 4. Patch multiple raster layers into a single raster

Patch raster tiles for lidar based, 6m res. DEM for Centennial Campus. 

In [ ]:
!g.region raster=el_D793_6m,el_D783_6m,el_D782_6m,el_D792_6m -p
!r.patch input=el_D793_6m,el_D783_6m,el_D782_6m,el_D792_6m output=elevlidD_6m
!r.colors elevlidD_6m raster=elevation

In [ ]:
# Create Map instance
elevlidD_6m_map = gj.Map(filename="patched_dem.png")
elevlidD_6m_map.d_rast(map="elevlidD_6m")
elevlidD_6m_map.d_vect(map="roadsmajor", color="grey")
# Display map
elevlidD_6m_map.show()

#### Question 4

**Using map algebra, Compute difference between the "elevlidD_6m" and "elevation" maps, assign it a diverging color ramp (e.g. "differences" provided by r.colors) and compute a summary statistics of the differences. Include the map and the statistics into your report and comment on the resulting pattern.**

### 5. Map Algebra

See [r.mapcalc manual page](https://grass.osgeo.org/grass-stable/manuals/r.mapcalc.html) for syntax and functions.

#### NDVI

Compute Normalized Difference Vegetation Index (NDVI).

In [ ]:
!g.region raster=lsat7_2002_40 -p
!r.mapcalc "ndvi1 = (lsat7_2002_40 - lsat7_2002_30) / (lsat7_2002_40 + lsat7_2002_30)"
!r.info -r ndvi1

In [ ]:
!g.region raster=lsat7_2002_40 -p
!r.mapcalc "ndvi2 = 1.0 * (lsat7_2002_40 - lsat7_2002_30) / (lsat7_2002_40 + lsat7_2002_30)"
!r.info -r ndvi2

In [ ]:
!g.region raster=lsat7_2002_40 -p
!r.mapcalc "ndvi3 = float(lsat7_2002_40 - lsat7_2002_30) / float(lsat7_2002_40 + lsat7_2002_30)"
!r.info -r ndvi3
!r.colors ndvi3 color=ndvi

In [ ]:
# Create Map instance
ndvi3_map = gj.Map(filename="ndvi3.png")
ndvi3_map.d_rast(map="ndvi3")
ndvi3_map.d_legend(raster="ndvi3")

# Display map
ndvi3_map.show()

Note that this is a simplified, map algebra example, for computing various vegetation indices in GRASS, we would use the [i.vi](https://grass.osgeo.org/grass-stable/manuals/i.vi.html) tool after performing atmospheric corrections.

#### Question 5

**Explain the difference between floating point and integer handling in ndvi1, ndvi2 and ndvi3 result.**

### 6. Working with if statements and null()

Create map of urban areas (high density and low density class) with 0 class elsewhere.

In [ ]:
!g.region raster=landuse96_28m -p
!r.mapcalc "urban1_30m = if(landuse96_28m == 1,1,0) + if(landuse96_28m == 2,2,0)"

Display the urban areas map.

In [ ]:
# Create Map instance
urban12_map = gj.Map(filename="urban12_nonurbanzero.png")
urban12_map.d_rast(map="urban1_30m")
urban12_map.d_legend(raster="urban1_30m", at="10,30,5,8")

# Display map
urban12_map.show()

Alternatively with logical 'or' operator and null elsewhere:

In [ ]:
!r.mapcalc "urban2_30m = if(landuse96_28m == 1 || landuse96_28m == 2,landuse96_28m,null())"

In [ ]:
# Create Map instance
urban2_30m_map = gj.Map(filename="urban2_nonurbannull.png")
urban2_30m_map.d_rast(map="urban2_30m")
urban2_30m_map.d_legend(raster="urban2_30m", at="10,30,5,8")

# Display map
urban2_30m_map.show()

#### Question 6

**What is the difference between the two urban raster maps?**

### 7. Optional - various additional useful tasks

#### 7A. Handling null values and creating a mask

1. Create mask for low lying developed areas (e.g. vulnerable to flooding) with elevation between 60 and 100m and land use 1 or 2. We could use this mask to exclude this areas from future high density urbanization in an urban growth simulation.
2. Set the region, display the input maps and create a MASK.

In [ ]:
!g.region raster=elevation -p
!r.mapcalc "low_elev_developed = if((elevation < 100 && elevation > 60) && (landuse96_28m == 1 || landuse96_28m == 2),1,null())"
!r.mask raster=low_elev_developed

Command [r.mask](https://grass.osgeo.org/grass-stable/manuals/r.mask.html) creates a raster map "MASK" in your mapset.

Display watersheds to see the mask effect:

In [ ]:
# Create Map instance
basin_masked_map = gj.Map(filename="basin_masked.png")
basin_masked_map.d_rast(map="basin_50K")

# Display map
basin_masked_map.show()

Disable the mask.

In [ ]:
!r.mask -r

Display basin_50K again to show that the mask was removed.

In [ ]:
# Create Map instance
basin_50K_map = gj.Map()
basin_50K_map.d_rast(map="basin_50K")
basin_50K_map.show()

#### 7B. Using coordinates of moving window in map algebra

Replace section of SRTM DSM with NED DEM elevation.

In [ ]:
!r.mapcalc "elev_combined = if(y() < 224274. && x() > 637455., elevation, elev_srtm_30m)"

##### Question 7B

**Try to explain how this r.mapcalc expression works.**

#### 7C. Tilted plane

Create a raster map representing tilted plane (e.g., geologic fault): 

In [ ]:
!g.region rural_1m -p
!r.mapcalc "tiltplane = 0.2*(0.1*row()+col())+50"
!r.mapcalc "tiltpl_under = if(tiltplane < elev_lid792_1m + 2,tiltplane,null())"

> GUI only: View the elevation surface and subsurface plane in 3D. Switch off all layers in the layer manager except for elev_lid792_1m and tiltpl_under. Change display to 3D view, adjust viewing position to a view from South. Save an image for your report.

#### 7D. Map subsets

Use map algebra to create map subsets.

Change region to the airphoto tile 792 and resolution 10m.

In [ ]:
!g.region raster=ortho_2001_t792_1m res=10 -ap

In [ ]:
# Create Map instance
ortho_2001_t792_1m_map = gj.Map()
ortho_2001_t792_1m_map.d_rast(map="ortho_2001_t792_1m")

# Display map
ortho_2001_t792_1m_map.show()

Create a subset of the map elevation for this subregion. 

In [ ]:
!r.mapcalc "elevation_792_10m = elevation"

In [ ]:
# Create Map instance
ortho_2001_t792_1m_map.d_rast(map="elevation_792_10m")

# Display map
ortho_2001_t792_1m_map.show()

#### 7E. Working with relative coordinates

Compute smoothed SRTM DEM using 3x3 moving window average.
Again, it is a good idea to remove the previously used map layers before we start to work on a new task.

In [ ]:
!g.region raster=elev_srtm_30m -p
!r.mapcalc "elev_srtm30m_smooth = (elev_srtm_30m[-1,-1] \
           + elev_srtm_30m[-1,0] + elev_srtm_30m[-1,1] \
           + elev_srtm_30m[0,-1] + elev_srtm_30m[0,0]  \
           + elev_srtm_30m[0,1]  + elev_srtm_30m[1,-1] \
           + elev_srtm_30m[1,0]  + elev_srtm_30m[1,1])/9."

Assign the resulting map the same color table as the original. Compare global statistics 

In [ ]:
!r.colors elev_srtm30m_smooth raster=elev_srtm_30m
!r.univar elev_srtm_30m
!r.univar elev_srtm30m_smooth

In [ ]:
# Create Map instance
elev_srtm_30m_map = gj.Map()
elev_srtm_30m_map.d_rast(map="elev_srtm_30m")
# Display map
elev_srtm_30m_map.show()

In [ ]:
# Create Map instance
elev_srtm30m_smooth_map = gj.Map()
elev_srtm30m_smooth_map.d_rast(map="elev_srtm30m_smooth")
# Display map
elev_srtm30m_smooth_map.show()